In [53]:
from nytimesarticle import articleAPI
api=articleAPI('0db8e23808f84821bed51f39d36d0787')
articles=api.search( q = 'Congress', begin_date = 20121231)

In [56]:
len(articles)

3

In [51]:
articles = api.search( q = 'Obama', 
     fq = {'headline':'', 'source':['Reuters','AP', 'The New York Times'], 'news_desk':['Politics','Business']}, 
     begin_date = 20121231 )

print articles


{u'status': u'OK', u'response': {u'docs': [{u'type_of_material': u'Series', u'blog': [], u'news_desk': u'Business', u'lead_paragraph': u'There\u2019s a relaxation in the approach to occupational safety and business is getting a bigger voice, while hard-won victories for safety advocates are being reversed.', u'headline': {u'main': u'Under Trump, Worker Protections Are Viewed With New Skepticism', u'kicker': u'Trump Rules', u'print_headline': u'Industry Given  A Louder Voice  On Job Safety', u'content_kicker': u'Trump Rules'}, u'abstract': None, u'print_page': 1, u'word_count': 1695, u'_id': u'593580717c459f24986e37f1', u'snippet': u'There\u2019s a relaxation in the approach to occupational safety and business is getting a bigger voice, while hard-won victories for safety advocates are being reversed....', u'source': u'The New York Times', u'slideshow_credits': None, u'web_url': u'https://www.nytimes.com/2017/06/05/business/under-trump-worker-protections-are-viewed-with-new-skepticism.h

In [54]:
len(articles)

3

In [2]:
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['id'] = i['_id']
        if i['abstract'] is not None:
            dic['abstract'] = i['abstract'].encode("utf8")
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['desk'] = i['news_desk']
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        dic['section'] = i['section_name']
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        dic['source'] = i['source']
        dic['type'] = i['type_of_material']
        dic['url'] = i['web_url']
        dic['word_count'] = i['word_count']
        # locations
        locations = []
        for x in range(0,len(i['keywords'])):
            if 'glocations' in i['keywords'][x]['name']:
                locations.append(i['keywords'][x]['value'])
        dic['locations'] = locations
        # subject
        subjects = []
        for x in range(0,len(i['keywords'])):
            if 'subject' in i['keywords'][x]['name']:
                subjects.append(i['keywords'][x]['value'])
        dic['subjects'] = subjects   
        news.append(dic)
    return(news) 
    

In [70]:
def get_articles(date,query):
    #'''
    #This function accepts a year in string format (e.g.'1980')
    #and a query (e.g.'Amnesty International') and it will 
    #return a list of parsed articles (in dictionaries)
    #for that year.
    #'''
    all_articles = []
    for i in range(1,4): #NYT limits pager to first 100 pages. But rarely will you find over 100 pages of results anyway.
        articles = api.search(q = query,
               fq = {'source':['Reuters','AP', 'The New York Times']},
               begin_date = date + '0101',
               end_date = date + '1231',
               desk='Politics',
               sort='oldest',
               page = str(i))
        if len(articles)>=2:
            articles = parse_articles(articles)
            all_articles = all_articles+articles
        else: return(all_articles)
    return(all_articles)

In [71]:
parse_articles(articles)

[{'date': u'2014-06-19',
  'desk': u'U.S.',
  'headline': 'Gay G.O.P. Candidate Begins Ad Campaign',
  'id': u'53a38b9d38f0d82561c762fe',
  'locations': [u'New Hampshire'],
  'section': u'U.S.',
  'snippet': 'Dan Innis, a gay Republican from New Hampshire, announced his candidacy for the House of Representatives in an advertisement....',
  'source': u'The New York Times',
  'subjects': [u'Political Advertising', u'Homosexuality and Bisexuality'],
  'type': u'Video',
  'url': u'https://www.nytimes.com/video/us/100000002951112/gay-gop-candidate-dan-innis-begins-ad-campaign.html',
  'word_count': u'19'},
 {'date': u'2017-05-08',
  'desk': u'OpEd',
  'headline': 'A Disaster Wrapped in Victory',
  'id': u'59101c7f7c459f24986de421',
  'locations': [],
  'section': u'Opinion',
  'snippet': 'With the health care vote, the House G.O.P. has made the rest of its agenda harder to achieve....',
  'source': u'The New York Times',
  'subjects': [u'Health Insurance and Managed Care',
   u'United State

In [72]:
aa=get_articles('2016', 'Obama')

In [74]:
len(aa)

10

In [14]:
import urllib2
import json
import datetime
import time
import sys
import argparse
import logging
from urllib2 import HTTPError
 
# helper function to iterate through dates
def daterange( start_date, end_date ):
    if start_date <= end_date:
        for n in range( ( end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )
 
# helper function to get json into a form I can work with       
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input
 
# helpful function to figure out what to name individual JSON files        
def getJsonFileName(date, page, json_file_path):
    json_file_name = ".".join([date,str(page),'json'])
    json_file_name = "".join([json_file_path,json_file_name])
    return json_file_name
 
# helpful function for processing keywords, mostly    
def getMultiples(items, key):
    values_list = ""
    if len(items) > 0:
        num_keys = 0
        for item in items:
            if num_keys == 0:
                values_list = item[key]                
            else:
                values_list =  "; ".join([values_list,item[key]])
            num_keys += 1
    return values_list
 
# get the articles from the NYTimes Article API    
def getArticles(date, api_key, json_file_path):
    # LOOP THROUGH THE 101 PAGES NYTIMES ALLOWS FOR THAT DATE
    for page in range(101):
        try:
            request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=" + date + "&end_date=" + date + "&page=" + str(page) + "&api-key=" + api_key
            response = urllib2.urlopen(request_string)
            content = response.read()
            if content:
                articles = convert(json.loads(content))
                # if there are articles here
                if len(articles["response"]["docs"]) >= 1:
                    json_file_name = getJsonFileName(date, page, json_file_path)
                    json_file = open(json_file_name, 'w')
                    json_file.write(content)
                    json_file.close()
                # if no more articles, go to next date
                else:
                    return
            # else:
            #     break
            time.sleep(3)
        except HTTPError as e:
            logging.error("HTTPError on page %s on %s (err no. %s: %s) Here's the URL of the call: %s", page, date, e.code, e.reason, request_string)
        except: 
            logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
            continue
 
# parse the JSON files you stored into a tab-delimited file
def parseArticles(date, csv_file_name, json_file_path):
    for file_number in range(101):
        # get the articles and put them into a dictionary
        try:
            file_name = getJsonFileName(date,file_number, json_file_path)
            in_file = open(file_name, 'r')
            articles = convert(json.loads(in_file.read()))
            in_file.close()
        except IOError as e:
			logging.error("IOError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
			continue
 
        # if there are articles in that document, parse them
        if len(articles["response"]["docs"]) >= 1:  
            # open the CSV for appending
            try:
                out_file = open(csv_file_name, 'ab')
            except IOError as e:
    			logging.error("IOError: %s %s", date, file_number, e.errno, e.strerror)
    			continue
 
            # loop through the articles putting what we need in a CSV   
            try:
                for article in articles["response"]["docs"]:
                    # if (article["source"] == "The New York Times" and article["document_type"] == "article"):
                    keywords = ""
                    keywords = getMultiples(article["keywords"],"value")
 
                    # should probably pull these if/else checks into a module
                    variables = [
                        article["pub_date"], 
                        keywords, 
                        str(article["headline"]["main"]).decode("utf8").replace("\n","") if "main" in article["headline"].keys() else "", 
                        str(article["source"]).decode("utf8") if "source" in article.keys() else "", 
                        str(article["document_type"]).decode("utf8") if "document_type" in article.keys() else "", 
                        article["web_url"] if "web_url" in article.keys() else "",
                        str(article["news_desk"]).decode("utf8") if "news_desk" in article.keys() else "",
                        str(article["section_name"]).decode("utf8") if "section_name" in article.keys() else "",
                        str(article["snippet"]).decode("utf8").replace("\n","") if "snippet" in article.keys() else "",
                        str(article["lead_paragraph"]).decode("utf8").replace("\n","") if "lead_paragraph" in article.keys() else "",
                        ]
                    line = "\t".join(variables)
                    out_file.write(line.encode("utf8")+"\n")
            except KeyError as e:
                logging.error("KeyError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
                continue
            except (KeyboardInterrupt, SystemExit):
                raise
            except: 
                logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
                continue
 
            out_file.close()
        else:
            break
 
# Main function where stuff gets done
 
def main():
    parser = argparse.ArgumentParser(description="A Python tool for grabbing data from the New York Times Article API.")
    parser.add_argument('-j','--json', required=True, help="path to the folder where you want the JSON files stored")
    parser.add_argument('-c','--csv', required=True, help="path to the file where you want the CSV file stored")
    parser.add_argument('-k','--key', required=True, help="your NY Times Article API key")
    # parser.add_argument('-s','--start-date', required=True, help="start date for collecting articles")
    # parser.add_argument('-e','--end-date', required=True, help="end date for collecting articles")
    args = parser.parse_args()
 
    json_file_path = args.json
    csv_file_name = args.csv
    api_key = args.key    
    start = datetime.date( year = 2013, month = 1, day = 1 )
    end = datetime.date( year = 2013, month = 1, day = 1 )
    log_file = "".join([json_file_path,"getTimesArticles_testing.log"])
    logging.basicConfig(filename=log_file, level=logging.INFO)
 
    logging.info("Getting started.") 
    try:
        # LOOP THROUGH THE SPECIFIED DATES
        for date in daterange( start, end ):
            date = date.strftime("%Y%m%d")
            logging.info("Working on %s." % date)
            getArticles(date, api_key, json_file_path)
            parseArticles(date, csv_file_name, json_file_path)
    except:
        logging.error("Unexpected error: %s", str(sys.exc_info()[0]))
    finally:
        logging.info("Finished.")
 
if __name__ == '__main__' :
    main()

usage: __main__.py [-h] -j JSON -c CSV -k KEY
__main__.py: error: argument -j/--json is required


SystemExit: 2

C:\Users\ewa\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
